---

# Bank

In [7]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch import nn

from time import time
from tqdm import tqdm

from sklearn.metrics import roc_auc_score, average_precision_score,auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, auc, precision_recall_curve
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import OneClassSVM as OCSVM

from sklearn.utils import shuffle

import pandas as pd
import os
import sys

seed = 1234
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

device = torch.device('cuda' if (torch.cuda.is_available()) else 'cpu')
print('[INFO] -> Using Device: ', device)

from load_data import get_Bank as get_data

[INFO] -> Using Device:  cuda


In [8]:
batch_size = 1024

BASE_PATH = '..'

DATASET_FILE = 'bank-additional-ful-nominal.arff'

SPACE_MODELS = './models'
SPACE_AUC = './auc'
SPACE_AUPRC = './auprc'

for n in (SPACE_MODELS, SPACE_AUC, SPACE_AUPRC):
    if not os.path.exists(n):
        os.mkdir(n)

DATASET = 'BANK'

PATH = os.path.join(BASE_PATH, 'Dataset', DATASET_FILE)

In [9]:
dataset = get_data(PATH, seed, show = True)

36548 normal records, 4640 anormal records
We use 2321 anomalous records
-----------------------------------------------------------------------------------------
There are 26383 records in training set
Training set is composed by 26383 normal records and 0 abnormal records
-----------------------------------------------------------------------------------------
There are 3131 records in validation set
Validation set is composed by 2551 normal records and 580 abnormal records
-----------------------------------------------------------------------------------------
There are 9354 records in test set
Test set is composed by 7614 normal records and 1740 abnormal records


#### Mean AUC and standard deviation after seven runs

In [10]:
def plot_pr_curve(precision, recall):
    plt.figure()
    plt.plot(recall, precision, marker='.')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.show()

def pr_auc(y_test, y_pred):
    precision, recall, _ = precision_recall_curve(y_test, y_pred)
    auc_score = auc(recall, precision)
    print(f'PR AUC: {auc_score:.2f}')
    plot_pr_curve(precision, recall)
    return auc_score

In [ ]:
n_runs = 20
auc_list = []
num_epochs = 200
seed = 42
pr_list = []

for i in range(n_runs):
    print(f'Iteration: {i+1}')
    dataset = get_data(PATH, seed*(i+1), show=False)
    dataset['y_test'].replace({0:-1}, inplace=True)
    
    ocsvm = OCSVM(gamma = 'auto', verbose = 2)
    ocsvm.fit(dataset['x_train'])
    y_pred = ocsvm.score_samples(dataset['x_test'])
    
    y_true = dataset['y_test']
    y_true = 1-(y_true+1)/2
    y_scores = -y_pred
    
    _auc = roc_auc_score(y_true, y_scores)
    pr = pr_auc(y_true, y_scores)
    print(f'AUC: {_auc:.2f}')
    
    auc_list.append(_auc)
    pr_list.append(pr)


def meanAUC_PRAUC(auc_list, pr_list, name):
    print('AUC:')
    print(auc_list)
    print('**************')
    print('PR AUC:')
    print(pr_list)
    print('**************')
    
    
    AUC_Frame = pd.DataFrame(auc_list, columns = [name])
    PR_AUC_Frame = pd.DataFrame(pr_list, columns = [name])
    
    AUC_Frame.to_csv(f'./auc/{name}.csv', index=False)
    PR_AUC_Frame.to_csv(f'./auc_PR/{name}.csv', index=False)
    
    AUC_Frame = list(AUC_Frame[name])

    N = len(AUC_Frame)
    mean_auc = np.mean(AUC_Frame)
    std_auc = np.std(AUC_Frame)
    std_error = std_auc / (np.sqrt(N))

    ci = 1.96 * std_error
    lower_bound = mean_auc - ci
    upper_bound = mean_auc + ci
    
    print('AUC')
    print(f'{mean_auc:.2f} +/- {ci:.2f}')
    print(f'95% confidence level, average auc would be between {lower_bound:.2f} and {upper_bound:.2f}')
    print('**************')
    
    PR_AUC_Frame = list(PR_AUC_Frame[name])

    N = len(PR_AUC_Frame)
    mean_auc = np.mean(PR_AUC_Frame)
    std_auc = np.std(PR_AUC_Frame)
    std_error = std_auc / (np.sqrt(N))

    ci = 1.96 * std_error
    lower_bound = mean_auc - ci
    upper_bound = mean_auc + ci
    
    print('PR AUC')
    print(f'{mean_auc:.2f} +/- {ci:.2f}')
    print(f'95% confidence level, average auc would be between {lower_bound:.2f} and {upper_bound:.2f}')

meanAUC_PRAUC(auc_list, pr_list, 'OCSVM_bank')

In [13]:
meanAUC_PRAUC(auc_list, pr_list, 'OCSVM_bank')

AUC:
[0.5954175082802702, 0.6097864943283546, 0.5995670030101838, 0.6056942519677907, 0.6109878883122137, 0.5917347128248327, 0.5983896874782992, 0.6020318363933348, 0.6026202111053746, 0.6038113774082225, 0.6013570358897251, 0.6127545220691466, 0.6019121612033489, 0.6111844409421241, 0.5988152118450888, 0.6115299176652809, 0.5922328876932692, 0.593779305514041, 0.603398533856266, 0.6019073304167459]
**************
PR AUC:
[0.2744101287471215, 0.28959769055389045, 0.2805381930665668, 0.29521487307583033, 0.2836857088756979, 0.2657404395648387, 0.2841256744501754, 0.2890849228089878, 0.281893623658347, 0.27980651284513813, 0.2760092294366398, 0.2911676384908633, 0.2882703078970984, 0.2965879191701368, 0.289936732253564, 0.3001486658684157, 0.2763122740522489, 0.2792253540251103, 0.28332816006114614, 0.27798842781412597]
**************
AUC
0.60 +/- 0.00
95% confidence level, average auc would be between 0.60 and 0.61
**************
PR AUC
0.28 +/- 0.00
95% confidence level, average auc w

---